# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [269]:
import pandas as pd
import numpy as np
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

#### 2)Conte o número de valores missing por variável

In [270]:
# 2) seu código aqui

sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

#### 3)     Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:

['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 

Refaça a contagem de valores missings. 

In [271]:
# 3) seu código aqui
# sinasc_v1: Selecionando as colunas

sinasc_v1 = sinasc [['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

sinasc_v1.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [272]:
contagem_linhas_APGAR5 = sinasc_selec['APGAR5'].count()

print (contagem_linhas_APGAR5)

26925


#### 4) Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings. 

In [273]:
# 4) seu código aqui
# sinasc_v2: retirando os valores nao preenchidas da coluna APGAR5

sinasc_v2 = sinasc_selec.dropna(subset=['APGAR5'])
sinasc_v2.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [274]:
contagem_linhas_APGAR5_dr = sinasc_selec_dr['APGAR5'].count()

print (contagem_linhas_APGAR5_dr)

26925


#### 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9.

In [275]:
# 5) seu código aqui
# sinasc_v3: Substituindo o não preenchido em 'ESTCIVMAE' e 'CONSULTAS' pelo 9

sinasc_v3 = sinasc_v2.fillna({'ESTCIVMAE':9, 'CONSULTAS':9})

sinasc_v3.isna().sum()


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

#### 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero. 

In [276]:
# 6) Seu código aqui
# sinasc_v4: Substituindo o não preenchido em 'QTDFILVIVO' pelo valor 0

sinasc_v4 = sinasc_v3.fillna({'QTDFILVIVO':0})

sinasc_v4.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

#### 7) Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui. 

In [277]:
# 7) seu código aqui
# sinasc_v5: Substituindo 'ESCMAE', 'GESTACAO' e 'GRAVIDEZ'.
# Substitui os campo 'ESCMAE' e 'GRAVIDEZ' por 9 (Ignorado), pois não 
# interfere nas próximas analises que serão realizadas.
# Substitui o campo 'GESTAÇÃO' POR 2, pois o pelo que encontrei, a 
# menor idade gestacional que um feto pode nascer e sobreviver é 22 semanas.

sinasc_v5 = sinasc_v4.fillna({'ESCMAE':9, 'GESTACAO':5, 'GRAVIDEZ':9})

sinasc_v5.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

#### 8)     O Apgar possui uma classificação indicando se o bebê passou por asfixia:

    Entre 8 e 10 está em uma faixa 'normal'.
    Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
    Entre 4 e 5 significa 'asfixia moderada'.
    Entre 0 e 3 significa 'asfixia severa'.

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

In [278]:
sinasc_v5.dtypes
len(sinasc_v5)

26925

In [279]:
# 8) seu código aqui

categoria = [
    (sinasc_v5['APGAR5'] >= 8) & (sinasc_v5['APGAR5'] <= 10),
    (sinasc_v5['APGAR5'] >= 6) & (sinasc_v5['APGAR5'] <= 7),
    (sinasc_v5['APGAR5'] >= 4) & (sinasc_v5['APGAR5'] <= 5),
    (sinasc_v5['APGAR5'] >= 0) & (sinasc_v5['APGAR5'] <= 3)
]

CATEGORIZAÇÃO = ['normal', 'asfixia leve', 'asfixia moderada', 'asfixia severa']

sinasc_v5['CATEGORIZAÇÃO'] = np.select(categoria, Categorizações)

print (sinasc_v5)

       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5 CATEGORIZAÇÃO  
0      37 a 41 semanas    Única          4    10.0        normal  
1      37 a 41 semanas    Única          4     9.

#### 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

In [283]:
# 9) seu código aqui


sinasc_v5.index.names = ['loc_nasc', 'idade_mae', 'est_civ_mae', 
                         'esc_mae', 'qtd_fil_vivo', 'gestação', 
                         'gravidez', 'consultas', 'apgar_5']

sinasc_v5.head()

CATEGORIZAÇÃO
loc_nasc idade_mae est_civ_mae esc_mae         qtd_fil_vivo gestação        gravidez consultas apgar_5              
1        19        5.0         8 a 11 anos     0.0          37 a 41 semanas Única    4         10.0           normal
         29        2.0         8 a 11 anos     1.0          37 a 41 semanas Única    4         9.0            normal
         37        9.0         8 a 11 anos     2.0          37 a 41 semanas Única    4         10.0           normal
         30        5.0         12 anos ou mais 0.0          37 a 41 semanas Única    3         10.0           normal
                   2.0         8 a 11 anos     1.0          37 a 41 semanas Única    4         10.0           normal